<a href="https://colab.research.google.com/github/kingamirasadi/Python-Pytorch-sklearn/blob/main/MVClusterin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install mat73


In [6]:
pip install torchcluster

In [15]:
import torch
import numpy as np
import mat73

In [16]:
mat_data = mat73.loadmat('/content/bbc_seg14of4.mat')


In [18]:
print((mat_data.keys()))

dict_keys(['data', 'labels'])


In [21]:
data_matrices = mat_data['data']
data_labels = mat_data['labels']

In [22]:
type(data_matrices)

list

In [23]:
X = []
for i in range(len(data_matrices)):
    tensor = torch.tensor(data_matrices[i]).squeeze()
    X.append(torch.transpose(tensor , 0,1 ))


In [ ]:
counts , unique_num = np.unique(data_labels, return_counts=True)
print(len(unique_num))

In [25]:
X[0].shape

torch.Size([4659, 145])

In [26]:
X[1].shape

torch.Size([4633, 145])

In [27]:
X[2].shape

torch.Size([4665, 145])

In [28]:
X[3].shape

torch.Size([4684, 145])

In [29]:
alpha = torch.tensor([0.1 , 0.2 , 0.3])
beta = torch.tensor([ 0.01 , 0.02 , 0.03])
landa = torch.tensor([0.001 , 0.002 , 0.003])

In [30]:
nu = len(data_matrices)

In [31]:
nu

4

In [32]:
omega =  torch.randn(1,nu)

In [33]:
omega

tensor([[ 0.6144,  0.1525, -2.0411, -1.1397]])

In [34]:
gamma = torch.randn(1,nu)

In [35]:
gamma

tensor([[-0.1076,  1.6383, -1.0786, -0.8724]])

In [36]:
n = len(data_labels)
k = len(unique_num)
F = torch.randn(n,k , dtype=torch.float)

In [37]:
Z = torch.eye(n,n)

In [ ]:
# H = torch.zeros((F.shape[0], F.shape[0]))

In [ ]:
# for i in range(F.shape[0]):
#     for j in range(i, F.shape[0]):
#         norm = torch.linalg.norm(F[i, :] - F[j, :], ord=2)
#         H[i, j] = norm
#         H[j, i] = norm

In [38]:
def laplacian_matrices(X):

    diagonal_matrices = []
    laplacian_matrices = []
    # creating similarity matrix(Gussian Kernel)
    for tensor in X:
        x_flat = tensor.view(tensor.size(0), -1)
        sigma = 1
        gaussian_sim = torch.exp(-torch.cdist(x_flat, x_flat, p=2.0) ** 2 / (2 * sigma ** 2))
        # sum of each similarity matrix row
        row_sums = torch.sum(gaussian_sim, dim=1)
        new_matrix = torch.diag(row_sums)
        diagonal_matrices.append(new_matrix)

        # Calculate the Laplacian matrix
        lap = new_matrix - gaussian_sim
        laplacian_matrices.append(lap)

    return laplacian_matrices

In [39]:
lap_list = laplacian_matrices(X)

In [40]:
for tensor in lap_list:
    eigenvalues, eigenvectors = torch.linalg.eigh(tensor)
    sorted_indices = torch.argsort(eigenvalues)
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    print(f"Eigenvalues for tensor:\n{eigenvalues}")
    print(f"Eigenvectors for tensor:\n{eigenvectors}")

Eigenvalues for tensor:
tensor([-1.6003e-12, -1.1999e-12, -6.3402e-13,  ...,  3.2032e+03,
         3.2032e+03,  3.2032e+03], dtype=torch.float64)
Eigenvectors for tensor:
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-6.9486e-15, -3.0995e-14,  2.0993e-15,  ..., -2.6515e-19,
         -2.8529e-17,  6.6848e-18],
        [ 1.5972e-14, -8.1330e-15, -1.9991e-16,  ..., -8.8481e-18,
          5.3200e-18,  2.1758e-17],
        ...,
        [ 6.4887e-15, -6.4345e-15, -1.1407e-14,  ..., -2.0343e-02,
         -6.6536e-03,  5.6497e-03],
        [ 6.4850e-15, -6.4068e-15, -1.1397e-14,  ...,  3.5707e-05,
         -1.4122e-02, -2.6715e-02],
        [ 6.5136e-15, -6.4111e-15, -1.1398e-14,  ...,  7.7069e-03,
          2.0734e-02,  2.0336e-03]], dtype=torch.float64)
Eigenvalues for tensor:
tensor([-1.4391e-11, -2.9168e-12, -1.0975e-12,  ...,  3.2111e+03,
         3.2111e+03,  3.2111e+03], dtype=torch.float64)
Eigenvectors for tensor:
tenso